In [ ]:
import pandas as pd
from datetime import datetime
import time
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import os
import glob


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from datetime import datetime
import time
import pandas as pd

service = Service(executable_path="C:/Windows/chromedriver.exe")
options = Options()
options.add_argument('--window-size=1920,1080')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0")
driver = webdriver.Chrome(service=service, options=options)

query = "Genshin_Impact"
target_year = 2023
max_results = 200
max_pages = 50
base_url = f"https://old.reddit.com/search/?q={query}&sort=relevance&t=all"

all_posts = []
seen_urls = set()
count = 0
after = None
page = 0

print(f"\nScraping up to {max_results} posts from {target_year} for query: '{query}'")
driver.get(base_url)
time.sleep(5)

def is_target_year(dt_text):
    try:
        dt = datetime.strptime(dt_text, "%Y-%m-%dT%H:%M:%S+00:00")
        return dt.year == target_year, dt
    except:
        return False, None

while len(all_posts) < max_results and page < max_pages:
    print(f"\nPage {page + 1} — Collected: {len(all_posts)} posts")

    posts = driver.find_elements(By.CSS_SELECTOR, "div.search-result")
    last_post_id = None

    for post in posts:
        try:
            title_elem = post.find_element(By.CSS_SELECTOR, "a.search-title")
            time_elem = post.find_element(By.CSS_SELECTOR, "span.search-time > time")
            url = title_elem.get_attribute("href")
            title = title_elem.text.strip()
            time_iso = time_elem.get_attribute("datetime")

            subreddit_elem = post.find_element(By.CSS_SELECTOR, "a.search-subreddit-link")
            subreddit = subreddit_elem.text.strip().replace("r/", "")

            full_id = post.get_attribute("data-fullname")

            if not url or url in seen_urls:
                continue
            seen_urls.add(url)

            valid, dt = is_target_year(time_iso)
            if not valid:
                try:
                    other_dt = datetime.strptime(time_iso, "%Y-%m-%dT%H:%M:%S+00:00")
                    print(f"Skipped non-{target_year} post: {other_dt.year} - {title[:50]}")
                except:
                    pass
                continue

            all_posts.append({
                "Subreddit": subreddit,
                "Year": dt.year,
                "Month": dt.month,
                "Text": title,
                "Created_utc": dt.strftime('%Y-%m-%d %H:%M:%S'),
            })
            print(f"[{dt.date()}] r/{subreddit} - {title[:60]}")

            if full_id:
                last_post_id = full_id

            if len(all_posts) >= max_results:
                break

        except Exception:
            continue

    if not last_post_id:
        print("No post ID found. Ending early.")
        break

    page += 1
    count += len(posts)
    after = last_post_id
    next_url = f"https://old.reddit.com/search/?q={query}&sort=relevance&t=all&count={count}&after={after}"

    print(f"Next: {next_url}")
    driver.get(next_url)
    time.sleep(4)

# --- Save results ---
driver.quit()
df = pd.DataFrame(all_posts)
output_file = f"{query.lower()}_{target_year}_posts.csv"
df.to_csv(output_file, index=False, encoding='utf-8')
print(f"\nScraped {len(df)} posts from {target_year}. Saved to {output_file}")



Scraping up to 200 posts from 2023 for query: 'Genshin_Impact'

Page 1 — Collected: 0 posts
Skipped non-2023 post: 2025 - Teyvat Chapter Interlude Teaser: The Gods' Limits 
Skipped non-2023 post: 2025 - Behind the Scenes of Genshin Impact "Let's Start W
Skipped non-2023 post: 2025 - Another "Genshin Impact players can't read" moment
Skipped non-2023 post: 2025 - Youtuber who used genshin impact music has apologi
Skipped non-2023 post: 2025 - Chinese players meme about what Genshin Impact wou
Skipped non-2023 post: 2025 - Nod-Krai Preview Teaser - "Prelude to Moonlight" |
Skipped non-2023 post: 2025 - Genshin Impact with more QOL Ideas!!!
Skipped non-2023 post: 2024 - Genshin Impact Loses Players' Voice Award from TGA
Skipped non-2023 post: 2024 - Genshin Impact Reddit Survey Results!
Skipped non-2023 post: 2024 - Ignition Teaser: A Name Forged in Flames | Genshin
Skipped non-2023 post: 2025 - Character Trailer - "Skirk: Lament of a Ruined Wor
Skipped non-2023 post: 2024 - Character Tr

In [ ]:
df = pd.read_csv(f"{query}_{target_year}_posts.csv")
df = df[df['Subreddit'].str.contains(query, case=False, na=False)]
df['Year'] = pd.to_datetime(df['Created_utc']).dt.year
df['Month'] = pd.to_datetime(df['Created_utc']).dt.month
df = df[["Subreddit", "Year", "Month", "Text", "Created_utc"]]
df.to_csv(f"{query}-{target_year}.csv", index=False)

In [ ]:
folder_path = f"{target_year} data"
output_file = f"merged_{target_year}data.csv"

csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
dfs = []

for file in csv_files:
    df = pd.read_csv(file)
    df.columns = [col.lower() for col in df.columns]

    if 'year' in df.columns:
        df = df[df['year'] == 2024]
    df = df.head(200)
    if not df.empty:
        dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)
merged_df.to_csv(output_file, index=False, encoding='utf-8')


✅ Merged 9 files into 'merged_2024data.csv' with 428 total rows (max 200 from each).
